In [1]:
from dataset import *
from architectures import *
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(7)

In [12]:
from torch.autograd import Variable

In [2]:
me = MazeExplorer(maze_size=(64,64), nbr_instances=10, 
                  difficulty='hard', nbr_trajectories=20,
                  alpha=1)

100%|██████████| 5/5 [00:00<00:00, 377.76it/s]

Creating MazeExplorer dataset of difficulty hard
Using Prims generator
Using shortest path solver



100%|██████████| 5/5 [00:00<00:00, 252.18it/s]


Dataset generated... 220 available instances


In [3]:
net = ConvNet()

In [4]:
# Get set of train/test INDEXES (hacking SKlean train_test_split)
x = np.arange(0, me.len())

train_indexes, test_indexes, _, _ = train_test_split(x, x, 
                                                    test_size=0.33, 
                                                    random_state=7)


In [5]:
# Wrap transforming function to dataset object
t_me = TransformedDataset(me, xy_transform=xy_transform)

# Get training/test subsets
train_me = Subset(t_me, indices=train_indexes)
test_me = Subset(t_me, indices=test_indexes)

# Configure dataloaders
batch_size = 1
num_workers = 1

trainloader = DataLoader(train_me, shuffle=True, 
                          batch_size=batch_size, 
                          num_workers=num_workers, 
                          pin_memory=True)


testloader = DataLoader(test_me, shuffle=True, 
                         batch_size=batch_size, 
                         num_workers=1, 
                         pin_memory=True)


In [28]:
#from torch.utils.data.sampler import SubsetRandomSampler

#Training
#n_training_samples = 20000
#train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
#n_val_samples = 5000
#val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
#n_test_samples = 5000
#test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))
#view raw

In [14]:
for batch in trainloader:
    print(batch)
    break

KeyboardInterrupt: 

In [29]:
from torch.optim import Adam

In [30]:
# Init losses
rmse = RMSELoss()
graphic_loss = GraphicLoss()

# Set optimizer (Adam)
optimizer = Adam(net.parameters(), lr=1e-3)

epochs = 2

In [15]:
torch.rand((1,64,64))[0].shape

torch.Size([64, 64])

In [31]:
for data, target in trainloader:
    print(data)
    print(target)
    break 

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x1c3df756a0>>
Traceback (most recent call last):
  File "/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/popen_fork.py", line 50, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


ValueError: too many values to unpack (expected 2)

In [25]:
#def train_model(epochs, net, criterion, optimizer, trainloader)
net.train()

for epoch in range(1, epochs+1):
    iters = 0
    for batch_idx, (data, target, path, goals, expert_flag) in enumerate(trainloader):
        # Convert X, Y to Torch variables
        grid = data   # save numpy version of maze grid
        print('batch taken')
        
        # Convert input and target to tensors
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)

        # Set gradient to zero
        optimizer.zero_grad()

        # Get network output
        output = net(data)
        print('inference made..')

        # Compute losses 
        if expert_flag:  # expert trajectory (imitation)
            loss = rmse(output, target)
            print('expert loss')

        else:  # non-expert trajectory (environment)
            # Maze walls loss (target, prediction)
            print('env loss')

            wall_loss = graphic_loss(grid, 
                                     torch.tensor(draw_solution_canvas(grid, 
                                                                       output.cpu().numpy())))
            # Path planning loss 
            path_loss = graphic_loss(path, 
                                     torch.tensor(draw_solution_canvas(path, 
                                                                       output.cpu().numpy())))
            path_loss = abs(wall_loss-path_loss)
            # Goals loss
            goal_loss = graphic_loss(goals, 
                                     torch.tensor(draw_goals_canvas(grid, 
                                                                    output.cpu().numpy())))

            # Compute final non-expert loss
            loss = wall_loss + path_loss + goal_loss  

        # Backpropagate loss & udpate gradient
        loss.backward()
        optimizer.step()

        # Print information
        iters += len(data)
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, iters, len(trainloader.dataset),
                100. * (batch_idx + 1) / len(trainloader), loss.data.item()), end='\r', flush=True)
    print("")
                                 
                                 

KeyboardInterrupt: 